In [42]:
import gzip
from collections import defaultdict
import random

def readGz(f):
	for l in gzip.open(f):
		yield eval(l)

def norm2(a,b):
	norm = 0
	for i in range(len(a)):
		norm += pow((a[i]-b[i]),2)
	return norm

def inner(x,y):
	return sum([x[i]*y[i] for i in range(len(x))])


In [43]:
itemIDmap = defaultdict(int)
userIDmap = defaultdict(int)
Iu = defaultdict(list)
Ui = defaultdict(list)
uid = 0
iid = 0
Ratings = defaultdict(float)

print "Reading Data..."
iter = 0
for l in readGz("R:/UCSD/Fall'15/CSE 255/Assignment 1/assignment1/train_900000.json.gz"):
	if l == "":
		continue
	if iter % 100000 == 0:
		print iter
	user,item = l['reviewerID'],l['itemID']
	Ratings[(user,item)] = l['rating']
	Iu[user].append(item)
	Ui[item].append(user)
	if user not in userIDmap:
		userIDmap[user] = uid
		uid += 1
	if item not in itemIDmap:
		itemIDmap[item] = iid
		iid += 1
	iter += 1
print "Done"
print len(Iu)
print len(Ui)
print len(Ratings)
print len(userIDmap)
print len(itemIDmap)


Reading Data...
0
100000
200000
300000
400000
500000
600000
700000
800000
Done
35736
37801
900000
35736
37801


In [44]:
Ratings_back = Ratings.copy()

In [54]:
Ratings = Ratings_back.copy()

In [66]:
avg = defaultdict(int)

for i in Ui:
    sum_i = sum([Ratings[(u,i)] for u in Ui[i]])
    avg[i] = sum_i*1.0/len(Ui[i])
    for u in Ui[i]:
        Ratings[(u,i)] -= avg[i]

In [67]:
print [Ratings[(u,'I939598332')] for u in Ui['I939598332']]
#[Ratings[u,'I939598332'] for u in Ui['I939598332']]
print Ratings[('U264510815', 'I939598332')]
print Ratings_back[('U264510815', 'I939598332')]

[0.7105263157894735, 0.7105263157894735, 0.7105263157894735, -1.2894736842105265, 0.7105263157894735, 0.7105263157894735, 0.7105263157894735, 0.7105263157894735, -1.2894736842105265, 0.7105263157894735, 0.7105263157894735, -1.2894736842105265, -0.28947368421052655, 0.7105263157894735, 0.7105263157894735, -2.2894736842105265, -0.28947368421052655, 0.7105263157894735, -0.28947368421052655, -0.28947368421052655, -0.28947368421052655, -0.28947368421052655, 0.7105263157894735, -0.28947368421052655, -0.28947368421052655, 0.7105263157894735, -0.28947368421052655, 0.7105263157894735, -0.28947368421052655, -1.2894736842105265, 0.7105263157894735, -2.2894736842105265, 0.7105263157894735, -0.28947368421052655, -0.28947368421052655, 0.7105263157894735, 0.7105263157894735, -0.28947368421052655]
0.710526315789
5.0


In [68]:
def f(Ratings, theta, x, lam):
    obj = 0
    for r in Ratings:
        obj += pow((Ratings[r] - inner(theta[r[0]], x[r[1]])),2)
    obj /= 2.0
    lamx = 0
    for i in x:
        lamx += sum([pow(x[i][k],2) for k in range(len(x[i]))])
    lamx *= lam/2.0
    lamt = 0
    for i in theta:
        lamt += sum([pow(theta[i][k],2) for k in range(len(theta[i]))])
    lamt *= lam/2.0
    return obj+lamx+lamt

In [69]:
def update_x(Ratings, theta, x, lam, alpha):
    for i in x:
        temp_x = [0]*len(x[i])
        for k in x[i]:
            temp_x[k] = sum([(Ratings[(u,i)] - inner(theta[u], x[i]))*theta[u][k] for u in Ui[i]]) + lam*x[i][k]
        x[i] = x[i] - alpha*temp_x
    return x[i]

In [70]:
def update_theta(Ratings, theta, x, lam, alpha):
    for u in theta:
        temp_t = [0]*len(theta[u])
        for k in theta[u]:
            temp_t[k] = sum([(Ratings[(u,i)] - inner(theta[u], x[i]))*x[i][k] for i in Iu[u]]) + lam*theta[u][k]
        theta[u] = theta[u] - alpha*temp_u
    return theta[i]

In [71]:
def get_vec(theta, x):
    vec = [0]*(2*len(theta))
    for u in theta:
        vec[userIDmap[u]] = theta[u]
    for i in x:
        vec[itemIDmap[i]] = x[i]
    return vec

In [72]:
flen = 3
x = defaultdict(list)
theta = defaultdict(list)
for i in itemIDmap:
    x[i] = [random.random() for k in range(flen)]

for u in userIDmap:
    theta[u] = [random.random() for k in range(flen)]

In [ ]:
iter = 0
lam = 5
alpha = 0.002
prev_J = f(Ratings, theta, x, lam)
while 1:
    x = update_x(Ratings, theta, x, lam, alpha)
    theta = update_theta(Ratings, theta, x, lam, alpha)
    J = f(Ratings, theta, x, lam)
    iter += 1
    if J == prev_J or J - prev_J < 1e-10 or iter == 500:
        break
mse = 0
    gui = ()
    for l in lst:
        u,i = l['reviewerID'],l['itemID']
        rate = l['rating']
        predict = inner(theta[u],x[i]) + avg[i]
        mse += pow((predict - rate),2)
    print (lam,mse/(len(lst)))